# LightGBM v1

### Imports

In [20]:
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

### Funciones

In [5]:
def normalizar_texto(descripcion):
    
    descripcion = descripcion.replace(',', ' ')
    descripcion = descripcion.replace('.', ' ')
    descripcion = descripcion.replace('<', ' ')
    descripcion = descripcion.replace('>', ' ')
    descripcion = descripcion.replace('\n', ' ')
    descripcion = descripcion.replace('\strong', ' ')
    descripcion = descripcion.replace('\bold', ' ')
    descripcion = descripcion.lower()
    
    return descripcion

def contar_palabras(titulo, palabras):
    
    contadas = 0
    
    for palabra in palabras:
        spliteado = titulo.split()
        contadas += spliteado.count(palabra)
        
    return contadas

### Reads
train.csv -> train
<br>
test.csv -> test

In [6]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

### Preparo el modelo 

**Feature Direción:**

In [7]:
DIRINVALIDAS = set(['Calle','domicilio conocido','calle','CALLE','--','Sin nombre ','1','.','-','x #x','0','...','S/N'])

df = train

df.loc[df.direccion.isin(DIRINVALIDAS), 'direccion'] = 0
df.loc[df['direccion'] != 0, 'direccion'] = 1

df['direccion'].value_counts()

1    237759
0      2241
Name: direccion, dtype: int64

**Feature Titulo y Descripcion:**

In [8]:
df['titulo'] = df['titulo'].fillna(value = ".")
df['titulo'] = df['titulo'].apply(normalizar_texto)
df['descripcion'] = df['descripcion'].fillna(value = ".")
df['descripcion'] = df['descripcion'].apply(normalizar_texto)

df['calefaccion'] = df['titulo'].apply(contar_palabras, palabras=["calefaccion", "calefacción", "calefaccionado", "aireacondicinado",
                    "acondicionado", "estufa","chimenea"]) + df['descripcion'].apply(contar_palabras, palabras=["calefaccion", 
                    "calefacción", "calefaccionado", "aireacondicinado", "acondicionado", "estufa","chimenea"])

df['suite'] = df['titulo'].apply(contar_palabras, palabras=["suite"])+df['descripcion'].apply(contar_palabras, palabras=["suite"])

df['avenida'] = df['titulo'].apply(contar_palabras, palabras=["avenida", "av", "avenidas", "bulevar", 
               "boulevard", "paseo", "vía"])+df['descripcion'].apply(contar_palabras, palabras=["avenida", "av", 
                "avenidas", "bulevar","boulevard", "paseo", "vía"])

df['gim'] = df['titulo'].apply(contar_palabras, palabras=["gimnasio", "gimnásio", "entrenamiento", "gim", "gym", "fit",
            "ejercicio", "gimnasia","atletismo", "cancha"])+df['descripcion'].apply(contar_palabras, palabras=["gimnasio",
            "gimnásio", "entrenamiento", "gim", "gym", "fit","ejercicio", "gimnasia","atletismo", "cancha"])

df['cochera'] = df['titulo'].apply(contar_palabras, palabras=["cochera", "cocheras", "garage", 
                "garages", "garaje", "garajes"])+df['descripcion'].apply(contar_palabras, palabras=["cochera", 
                "cocheras", "garage","garages", "garaje", "garajes"])

df['ubicacion'] = df['titulo'].apply(contar_palabras, palabras=["ubicacion", "ubicación", "locacion", 
                "locación", "localizacion", "localización","ubicado", "ubicada", "centro", "centrico",
                "centrica", "céntrico", "céntrica","central"])+df['descripcion'].apply(contar_palabras, palabras=["ubicacion", "ubicación", "locacion", 
                "locación", "localizacion", "localización","ubicado", "ubicada", "centro", "centrico",
                "centrica", "céntrico", "céntrica","central"])

df['balcon'] = df['titulo'].apply(contar_palabras, palabras=["balcon", "balcón", "terraza", "palco", "mirador", 
            "balconcillo","azotea", "solana"])+df['descripcion'].apply(contar_palabras, palabras=["balcon", "balcón", 
            "terraza", "palco", "mirador", "balconcillo","azotea", "solana"])

df['camaraseg'] = df['titulo'].apply(contar_palabras, palabras=["camara", "cámara", "cámaras", "camaras","seguridad",
                "guardia","seguro","protegido"]) + df['descripcion'].apply(contar_palabras, palabras=["camara", "cámara",
                "cámaras", "camaras","seguridad","guardia","seguro","protegido"])

df['parque'] = df['titulo'].apply(contar_palabras, palabras=["parque", "plaza", "plazoleta", "glorieta","jardin",
            "jardín","patio"])+df['descripcion'].apply(contar_palabras, palabras=["parque", "plaza", "plazoleta", 
            "glorieta","jardin","jardín","patio"])

df['amoblado'] = df['titulo'].apply(contar_palabras, palabras=["muebles", "amoblado", "mueble","decorado",
                "listo"]) + df['descripcion'].apply(contar_palabras, palabras=["muebles", "amoblado", "mueble",
                "decorado","listo"])

df['bañera'] = df['titulo'].apply(contar_palabras, palabras=["bañera", "hidromasaje", "hidro", "tina",
            "jacuzzi","jacuzi","yacuzi"]) + df['descripcion'].apply(contar_palabras, palabras=["bañera", "hidromasaje", 
            "hidro", "tina","jacuzzi","jacuzi","yacuzi"])


df['estreno'] = df['titulo'].apply(contar_palabras, palabras=["nuevo", "nueva", "estrenar","estreno",
            "innovador"]) + df['descripcion'].apply(contar_palabras, palabras=["nuevo", "nueva", "estrenar","estreno",
            "innovador"])

df['transporte'] = df['titulo'].apply(contar_palabras, palabras=["subte", "subterraneo", "subterráneo","metro", 
                "estacion", "estación", "tren","subestacion", "subestación", "ferrocarril","metrobús", "metrobus", 
                "trolebus","trolebús", "bus", "bús"]) + df['descripcion'].apply(contar_palabras, palabras=["subte", 
                "subterraneo", "subterráneo","metro", "estacion", "estación", "tren","subestacion", "subestación", 
                "ferrocarril","metrobús", "metrobus", "trolebus","trolebús", "bus", "bús"])

df['pileta'] = df['titulo'].apply(contar_palabras, palabras=["piscina", "pileta", "nado"])+ df['descripcion'].apply(contar_palabras, palabras=["piscina", "pileta", "nado"])
                                       
df['lujo'] = df['titulo'].apply(contar_palabras, palabras=["lujo", "delujo", "deluxe", "delúxe", "lujosa", "lujoso", 
            "lujosas", "lujosos","exclusivo","vip"]) + df['descripcion'].apply(contar_palabras, palabras=["lujo", 
            "delujo","deluxe", "delúxe", "lujosa", "lujoso", "lujosas", "lujosos","exclusivo","vip"])

df['humilde'] = df['titulo'].apply(contar_palabras, palabras=["humilde", "economico", "economica", 
            "económico", "económica", "barata", "barato", "accesible", "baratillo",
            "baratilla", "rebajado", "ganga", "asequible", "módico", "módica","credito","crédito","oferta","oferton",
            "imperdible"]) + df['descripcion'].apply(contar_palabras, palabras=["humilde", "economico", "economica", 
            "económico", "económica", "barata", "barato", "accesible", "baratillo",
            "baratilla", "rebajado", "ganga", "asequible", "módico", "módica",
            "credito","crédito","oferta","oferton","imperdible"]) 

df['ventana'] = df['titulo'].apply(contar_palabras, palabras=["ventana", "ventanas", 
            "vista", "ventanal","vistas","cristal"]) + df['descripcion'].apply(contar_palabras, palabras=["ventana",
            "ventanas", "vista", "ventanal","vistas","cristal"])

df['nuevo'] = df['titulo'].apply(contar_palabras, palabras=["reciente", "recien", "recién", "nueva", "nuevo", "nuevas", 
            "nuevos", "estrenar"]) + df['descripcion'].apply(contar_palabras, palabras=["reciente", "recien", "recién", 
            "nueva", "nuevo", "nuevas", "nuevos", "estrenar"])

df['luz'] = df['titulo'].apply(contar_palabras, palabras=["luz", "luminoso", "luminosa","claridad", "luminiscencia", 
        "luminosidad", "iluminación","iluminacion"]) + df['descripcion'].apply(contar_palabras, palabras=["luz", 
        "luminoso", "luminosa","claridad", "luminiscencia","luminosidad", "iluminación","iluminacion"])

df['bueno'] = df['titulo'].apply(contar_palabras, palabras=["bueno", "buena", "buenas", "buenos","excelente", 
            "excelentes","increible","espectacular"]) + df['descripcion'].apply(contar_palabras, palabras=["bueno", 
            "buena", "buenas", "buenos","excelente", "excelentes","increible","espectacular"])

df['contable'] = df['titulo'].apply(contar_palabras, palabras=["precio"]) + df['descripcion'].apply(contar_palabras, palabras=["precio"])

#Nuevos Feat

df['agente'] = df['descripcion'].apply(contar_palabras, palabras=["inmobiliaria", "asesoria", "asesoría", "lider", "re/max", "remax"]) \
             + df['titulo'].apply(contar_palabras, palabras=["inmobiliaria", "asesoria", "asesoría", "lider", "re/max", "remax"])

df['garante'] = df['descripcion'].apply(contar_palabras, palabras=["garante", "garantía", "fiador", "garantizador", "avalista", "garantia",
                "defensor", "garantías", "garantes", "codeudor"]) + df['titulo'].apply(contar_palabras, palabras=["garante", 
                "garantía", "fiador", "garantizador", "avalista", "garantia", "defensor", "garantías", "garantes", "codeudor"])

df['finanza'] =  df['descripcion'].apply(contar_palabras, palabras=["credito", "crédito", "prestamo", "préstamo", "cuotas", "pagos", "hipotecario"\
                 ,"amortizable", "aptocredito", "aptocrédito", "apto-credito", "apto-crédito", "aptocredito", "aptocréditp", "ahorro"]) + \
                 df['titulo'].apply(contar_palabras, palabras=["credito", "crédito", "prestamo", "préstamo", "cuotas", "pagos", "hipotecario" \
                 ,"amortizable", "aptocredito", "aptocrédito", "apto-credito", "apto-crédito", "aptocredito", "aptocrédito", "ahorro"]) 

df['turismo'] =  df['descripcion'].apply(contar_palabras, palabras=["playa", "vacaciones", "descanso", "costa", "arena", "mar", "montaña", "monte",
                                                                   "paisaje", "orilla", "rambla", "turista", "turistas", "cordillera", "sierra"])\
                 + df['titulo'].apply(contar_palabras, palabras=["playa", "vacaciones", "descanso", "costa", "arena", "mar", "montaña", "monte",
                                                                   "paisaje", "orilla", "rambla", "turista", "turistas", "cordillera", "sierra"])

df['longitud_desc'] =  df['descripcion'].apply(lambda x: len(x.split()))

#Nuevas de V4

df['longitud_titulo'] = df['titulo'].apply(lambda x: len(x))

df['cant_!'] = df['titulo'].apply(lambda x: x.count('!')) 

df['tranquilidad'] = df['descripcion'].apply(contar_palabras, palabras=["armonía", "armonia", "tranquilo", "tranqui", "tranquilidad", \
                                                                        "paz", "calma", "calmo", "quietud"])\
                 + df['titulo'].apply(contar_palabras, palabras=["armonía", "armonia", "tranquilo", "tranqui", "tranquilidad", \
                                                                        "paz", "calma", "calmo", "quietud"])

df['reparacion'] = df['descripcion'].apply(contar_palabras, palabras=["reparación", "reparacion", "reparando", "reparar", \
                            "construcción", "construccion", "construyendo", "construllendo", "mantenimiento", "averiado", "averiada",
                                                                     "refaccion","refacción","refacciones"])\
                 + df['titulo'].apply(contar_palabras, palabras=["reparación", "reparacion", "reparando", "reparar", \
                            "construcción", "construccion", "construyendo", "construllendo", "mantenimiento", "averiado", "averiada"
                                                                     "refaccion","refacción","refacciones"])

df['mascotas'] = df['descripcion'].apply(contar_palabras, palabras=["mascotas", "mascota", "perros", "perro", "perra", \
                                                "gatos", "gato", "gata", "animal", "animales"])\
                 + df['titulo'].apply(contar_palabras, palabras=["mascotas", "mascota", "perros", "perro", "perra", \
                                                "gatos", "gato", "gata", "animal", "animales"])

df['accesibilidad'] = df['descripcion'].apply(contar_palabras, palabras=["rampa", "discapacitados", "discapacitado", \
                                                "discapacitada", "lisiado", "lisiada", "silla de ruedas", "lastimado", "heridos"])\
                 + df['titulo'].apply(contar_palabras, palabras=["rampa", "discapacitados", "discapacitado", \
                                                "discapacitada", "lisiado", "lisiada", "silla de ruedas", "lastimado", "heridos"])

df['normas'] =  df['descripcion'].apply(contar_palabras, palabras=["norma", "normas", "regla", "reglas", \
                                                "prohibido", "prohibida", "denegado", "denegada", "imposible", \
                                                "ilegal", "legal", "multa", "infraccion", "infracción"])\
                 + df['titulo'].apply(contar_palabras, palabras=["norma", "normas", "regla", "reglas", \
                                                "prohibido", "prohibida", "denegado", "denegada", "imposible", \
                                                "ilegal", "legal", "multa", "infraccion", "infracción"])

df['beneficios'] = df['descripcion'].apply(contar_palabras, palabras=["gratis", "free", "incluido", "incluye", \
                                                "agregado", "gratuito", "gratuitamente", \
                                                "regalo"])\
                 + df['titulo'].apply(contar_palabras, palabras=["gratis", "free", "incluido", "incluye", \
                                                "agregado", "gratuito", "gratuitamente", \
                                                "regalo"])

df['conexion'] = df['descripcion'].apply(contar_palabras, palabras=["wifi", "wi-fi", "internet", "conexión", "conexion", \
                                                                   "señal"])\
                 + df['titulo'].apply(contar_palabras, palabras=["wifi", "wi-fi", "internet", "conexión", "conexion", \
                                                                   "señal"])

df['servicios_desc'] = df['descripcion'].apply(contar_palabras, palabras = ["servicio", "servicios"])

df['metros_desc'] = df['descripcion'].apply(contar_palabras, palabras = ["metros","m2"])

df['acabados'] = df['descripcion'].apply(contar_palabras, palabras = ["acabados","acabado","terminacion","terminación"])

df['plusvalia'] = df['descripcion'].apply(contar_palabras, palabras = ["plusvalia", "plusvalía"])

#Nuevos de V5

df['cocina'] = df['descripcion'].apply(contar_palabras, palabras=['cocina', 'cocinas'])

df['alberca'] = df['descripcion'].apply(contar_palabras, palabras=['alberca'])

df['negacion'] = df['descripcion'].apply(contar_palabras, palabras = ["no"])

df['variospisos'] = df['descripcion'].apply(contar_palabras, palabras = ["escalera","escaleras",
                    "ascensor", "elevador", "escalinata", "gradas", "escalerilla"])

df['vestidor'] = df['descripcion'].apply(contar_palabras, palabras = ["vestidor"])

**Feature Metros:**

In [9]:
df['metrosdescubiertos'] = abs(df['metrostotales']-df['metroscubiertos'])
df['relacionmetros'] = df['metrostotales']/df['metroscubiertos']

In [10]:
def mi_split (df):

    X = df.drop(['precio'], axis = 1)
    Y = df['precio']
    x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.35)
    return x_train, x_test, y_train, y_test

from scipy.stats import uniform, randint
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn import preprocessing

def minmax(serie):
    
    return (serie-serie.min())/(serie.max()-serie.min())

def mean_target_encoding(train, nombrefeat, nombretarget):
    
    temp = train.groupby(nombrefeat)[nombretarget].transform(np.mean)
    train[nombrefeat + "_mean"]=(temp-temp.min())/(temp.max()-temp.min())
    
    return train

def mean_target_decoding(x_test, nombrefeat, x_train):
    
    nombrefeatmean = nombrefeat + "_mean"
    
    temp = x_train.loc[:,[nombrefeat,nombrefeatmean]]
    temp = temp.set_index(nombrefeat)
    temp = temp.drop_duplicates()
    temp = temp.T.squeeze()
    values = x_test[nombrefeat].map(temp)
    x_test[nombrefeatmean] = values 

    return x_test

def armar_set_2(train):
    
    #Puedo resolver de forma general las que son iguales para train y test
    #es decir, aquellas que no filtran informacion a los de validacion
    
    train = train.drop(['descripcion', 'titulo', 'direccion', 'id'], axis = 1)
    
    #Adiciono servicios
    train['servicios'] = train['piscina'] + train['gimnasio'] + train['usosmultiples']
    
    train['fecha'] = pd.to_datetime(train['fecha']).dt.year
    
    #"Normalizo" la antiguedad
    #train['antiguedad'] = minmax(train['antiguedad'])
    
    #Elimino los residuos
    #train = train.drop(['piscina', 'gimnasio','usosmultiples','escuelascercanas', 
                       # 'centroscomercialescercanos'], axis = 1)
    
    #Hago el split                    
    x_train, x_test, y_train, y_test = mi_split(train)
        
    x_train["precio"] = y_train
    x_test["precio"] = y_test
    
    
    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometro']=df1['precio']/df1['metrostotales']

    PMM = df1.groupby('idzona').agg({'preciometro': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometro": "precioMetroMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')


    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometroCub']=df1['precio']/df1['metroscubiertos']

    PMM = df1.groupby('idzona').agg({'preciometroCub': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroCub": "precioMetroCubMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')
    
    
    
    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometroDes']=df1['precio']/df1['metrosdescubiertos']

    PMM = df1.groupby('idzona').agg({'preciometroDes': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroDes": "precioMetroDesMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')
    
    
    #x = x_test.fillna(0)
    #x_test['primera_pred'] = (x['precioMetroCubMean']*x['metroscubiertos']+x['precioMetroMean']*x['metrostotales'])/2
    
    #x = x_train.fillna(0)
    #x_train['primera_pred'] = (x['precioMetroCubMean']*x['metroscubiertos']+x['precioMetroMean']*x['metrostotales'])/2

    
    #Calculo los mean target
    x_train_mean = mean_target_encoding(x_train, "provincia", "precio") 
    x_train_mean['precio'] = y_train
    x_train_mean = mean_target_encoding(x_train_mean, "tipodepropiedad", "precio")
    x_train_mean = mean_target_encoding(x_train_mean, "ciudad", "precio")
    x_train = mean_target_encoding(x_train_mean, "idzona", "precio")

    
    #Se los asigno a los test (NO LOS CALCULO CON ELLOS!!!!!!)
    x_test = mean_target_decoding(x_test, "provincia", x_train_mean)
    x_test = mean_target_decoding(x_test, "tipodepropiedad", x_train_mean)
    x_test = mean_target_decoding(x_test, "ciudad", x_train_mean)
    x_test = mean_target_decoding(x_test, "idzona", x_train_mean)
    
    backup = x_train_mean
    
    x_train = x_train_mean.drop(["precio","provincia","tipodepropiedad","ciudad"], axis=1)
    x_test.drop(["precio","provincia","tipodepropiedad", "ciudad"], axis=1, inplace = True)

    
    return x_train, x_test, y_train, y_test, backup

In [11]:
x_train, x_test, y_train, y_test, backup = armar_set_2(df)

y = y_train.values

x = x_train.values

y_test = y_test.values

x_test = x_test.values

# divido el df 

/home/belu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/belu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/belu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

### Creo LightGBM data containers

In [12]:
categorical_features = [c for c, col in enumerate(train.columns) if 'cat' in col]
train_data = lightgbm.Dataset(x, label=y, categorical_feature=categorical_features)
test_data = lightgbm.Dataset(x_test, label=y_test)


In [58]:
#Lo entreno

params = {
    'n_estimators' : 3000,
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l1', 'l2'},
    'learning_rate': 0.3,
    'feature_fraction': 0.9,
    'bagging_fraction': 1,
    'bagging_freq': 5,
    'min_child_samples': 326,
    'min_child_weight': 200.0,
    'reg_alpha': 1000,#50
    'reg_lambda': 0,
    'verbose': 0
}

model = lightgbm.train(params,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=5000,
                       early_stopping_rounds=100)

/home/belu/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/belu/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/belu/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1247: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	valid_0's l2: 3.24868e+12	valid_0's l1: 1.31305e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l2: 2.35232e+12	valid_0's l1: 1.08491e+06
[3]	valid_0's l2: 1.86852e+12	valid_0's l1: 937377
[4]	valid_0's l2: 1.58833e+12	valid_0's l1: 839186
[5]	valid_0's l2: 1.44517e+12	valid_0's l1: 781490
[6]	valid_0's l2: 1.32625e+12	valid_0's l1: 733566
[7]	valid_0's l2: 1.25634e+12	valid_0's l1: 704634
[8]	valid_0's l2: 1.1984e+12	valid_0's l1: 679981
[9]	valid_0's l2: 1.16378e+12	valid_0's l1: 664712
[10]	valid_0's l2: 1.12964e+12	valid_0's l1: 651022
[11]	valid_0's l2: 1.10542e+12	valid_0's l1: 641612
[12]	valid_0's l2: 1.08617e+12	valid_0's l1: 633666
[13]	valid_0's l2: 1.06488e+12	valid_0's l1: 627442
[14]	valid_0's l2: 1.05083e+12	valid_0's l1: 623171
[15]	valid_0's l2: 1.0417e+12	valid_0's l1: 619428
[16]	valid_0's l2: 1.03233e+12	valid_0's l1: 616111
[17]	valid_0's l2: 1.0215e+12	valid_0's l1: 612594
[18]	valid_0's l2: 1.01566e+12	valid_0's l1: 610281
[19]

[162]	valid_0's l2: 8.36807e+11	valid_0's l1: 546470
[163]	valid_0's l2: 8.3648e+11	valid_0's l1: 546272
[164]	valid_0's l2: 8.36378e+11	valid_0's l1: 546272
[165]	valid_0's l2: 8.36247e+11	valid_0's l1: 546258
[166]	valid_0's l2: 8.35814e+11	valid_0's l1: 546192
[167]	valid_0's l2: 8.35458e+11	valid_0's l1: 545976
[168]	valid_0's l2: 8.34784e+11	valid_0's l1: 545747
[169]	valid_0's l2: 8.34549e+11	valid_0's l1: 545672
[170]	valid_0's l2: 8.34519e+11	valid_0's l1: 545626
[171]	valid_0's l2: 8.34406e+11	valid_0's l1: 545601
[172]	valid_0's l2: 8.33848e+11	valid_0's l1: 545414
[173]	valid_0's l2: 8.33757e+11	valid_0's l1: 545392
[174]	valid_0's l2: 8.33724e+11	valid_0's l1: 545381
[175]	valid_0's l2: 8.33616e+11	valid_0's l1: 545341
[176]	valid_0's l2: 8.33373e+11	valid_0's l1: 545227
[177]	valid_0's l2: 8.33356e+11	valid_0's l1: 545203
[178]	valid_0's l2: 8.33136e+11	valid_0's l1: 545152
[179]	valid_0's l2: 8.33009e+11	valid_0's l1: 545056
[180]	valid_0's l2: 8.32927e+11	valid_0's l1: 5

[323]	valid_0's l2: 8.13492e+11	valid_0's l1: 539351
[324]	valid_0's l2: 8.13515e+11	valid_0's l1: 539396
[325]	valid_0's l2: 8.13537e+11	valid_0's l1: 539399
[326]	valid_0's l2: 8.13382e+11	valid_0's l1: 539381
[327]	valid_0's l2: 8.13396e+11	valid_0's l1: 539365
[328]	valid_0's l2: 8.13311e+11	valid_0's l1: 539363
[329]	valid_0's l2: 8.13173e+11	valid_0's l1: 539316
[330]	valid_0's l2: 8.13133e+11	valid_0's l1: 539268
[331]	valid_0's l2: 8.13049e+11	valid_0's l1: 539257
[332]	valid_0's l2: 8.13057e+11	valid_0's l1: 539294
[333]	valid_0's l2: 8.13072e+11	valid_0's l1: 539243
[334]	valid_0's l2: 8.12953e+11	valid_0's l1: 539227
[335]	valid_0's l2: 8.12875e+11	valid_0's l1: 539230
[336]	valid_0's l2: 8.12831e+11	valid_0's l1: 539220
[337]	valid_0's l2: 8.12634e+11	valid_0's l1: 539229
[338]	valid_0's l2: 8.12556e+11	valid_0's l1: 539230
[339]	valid_0's l2: 8.12451e+11	valid_0's l1: 539192
[340]	valid_0's l2: 8.12234e+11	valid_0's l1: 539118
[341]	valid_0's l2: 8.12169e+11	valid_0's l1: 

[478]	valid_0's l2: 8.07477e+11	valid_0's l1: 537651
[479]	valid_0's l2: 8.07482e+11	valid_0's l1: 537653
[480]	valid_0's l2: 8.07357e+11	valid_0's l1: 537661
[481]	valid_0's l2: 8.07324e+11	valid_0's l1: 537723
[482]	valid_0's l2: 8.0736e+11	valid_0's l1: 537733
[483]	valid_0's l2: 8.07414e+11	valid_0's l1: 537686
[484]	valid_0's l2: 8.07483e+11	valid_0's l1: 537687
[485]	valid_0's l2: 8.07308e+11	valid_0's l1: 537630
[486]	valid_0's l2: 8.07348e+11	valid_0's l1: 537627
[487]	valid_0's l2: 8.07502e+11	valid_0's l1: 537661
[488]	valid_0's l2: 8.07412e+11	valid_0's l1: 537591
[489]	valid_0's l2: 8.07498e+11	valid_0's l1: 537606
[490]	valid_0's l2: 8.0745e+11	valid_0's l1: 537678
[491]	valid_0's l2: 8.07345e+11	valid_0's l1: 537676
[492]	valid_0's l2: 8.0732e+11	valid_0's l1: 537689
[493]	valid_0's l2: 8.07386e+11	valid_0's l1: 537689
[494]	valid_0's l2: 8.07485e+11	valid_0's l1: 537732
[495]	valid_0's l2: 8.07437e+11	valid_0's l1: 537748
[496]	valid_0's l2: 8.07377e+11	valid_0's l1: 537

[637]	valid_0's l2: 8.04489e+11	valid_0's l1: 537168
[638]	valid_0's l2: 8.04478e+11	valid_0's l1: 537179
[639]	valid_0's l2: 8.04445e+11	valid_0's l1: 537099
[640]	valid_0's l2: 8.04372e+11	valid_0's l1: 537068
[641]	valid_0's l2: 8.04464e+11	valid_0's l1: 537118
[642]	valid_0's l2: 8.04315e+11	valid_0's l1: 537081
[643]	valid_0's l2: 8.04306e+11	valid_0's l1: 537115
[644]	valid_0's l2: 8.04359e+11	valid_0's l1: 537156
[645]	valid_0's l2: 8.04387e+11	valid_0's l1: 537143
[646]	valid_0's l2: 8.04362e+11	valid_0's l1: 537160
[647]	valid_0's l2: 8.04386e+11	valid_0's l1: 537207
[648]	valid_0's l2: 8.04458e+11	valid_0's l1: 537216
[649]	valid_0's l2: 8.04425e+11	valid_0's l1: 537199
[650]	valid_0's l2: 8.04627e+11	valid_0's l1: 537275
[651]	valid_0's l2: 8.04626e+11	valid_0's l1: 537286
[652]	valid_0's l2: 8.04576e+11	valid_0's l1: 537271
[653]	valid_0's l2: 8.04691e+11	valid_0's l1: 537351
[654]	valid_0's l2: 8.04634e+11	valid_0's l1: 537354
[655]	valid_0's l2: 8.04744e+11	valid_0's l1: 

In [61]:
prediccion = model.predict(x_test)

In [62]:
#Estimo MAE
print(mean_absolute_error(y_test, prediccion))

536807.6261399545


# gridsearch

In [45]:
params = {'boosting_type': 'gbdt',
          'max_depth' : -1,
          'objective': 'regression',
          'nthread': None, # Updated from nthread
          'num_leaves': 64,
          'learning_rate': 0.05,
          'max_bin': 512,
          'subsample_for_bin': 200,
          'subsample': 1,
          'subsample_freq': 1,
          'colsample_bytree': 0.8,
          'reg_alpha': 5,
          'reg_lambda': 10,
          'min_split_gain': 0.5,
          'min_child_weight': 1,
          'min_child_samples': 5,
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'binary_error'}

# creo los parametros para grid  
gridParams = {
    'learning_rate': [0.005],
    'n_estimators': [1,2],
    'num_leaves': [6,8,12,16],
    'boosting_type' : ['gbdt'],
    'objective' : ['regression'],
    'random_state' : [501], 
    'colsample_bytree' : [0.65, 0.66],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2,1.4],
    }

# pongo los parametros 
mdl = lgb.LGBMRegressor(boosting_type= 'gbdt',
          objective = 'regression',
          n_jobs = None, # Updated from 'nthread'
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'],
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'],
          subsample_freq = params['subsample_freq'],
          min_split_gain = params['min_split_gain'],
          min_child_weight = params['min_child_weight'],
          min_child_samples = params['min_child_samples'],
          scale_pos_weight = params['scale_pos_weight'])

#default:
mdl.get_params().keys()

# Armo grid
grid = GridSearchCV(mdl, gridParams,
                    verbose=0,
                    cv=4,
                    n_jobs=None)
# Enteno
grid.fit(x, y_train)

GridSearchCV(cv=4, error_score='raise-deprecating',
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_bin=512, max_depth=-1,
                                     min_child_samples=5, min_child_weight=1,
                                     min_split_gain=0.5, n_estimators=100,
                                     n_jobs=None, num_leaves=31,
                                     objective='binary', random_state=None,
                                     reg_alpha=0...
             iid='warn', n_jobs=None,
             param_grid={'boosting_type': ['gbdt'],
                         'colsample_bytree': [0.65, 0.66],
                         'learning_rate': [0.005], 'n_estimators': [1, 2],
                         'num_leaves': [6, 8, 12, 16],
                         'objective': ['r

In [48]:
grid.best_params_

{'boosting_type': 'gbdt',
 'colsample_bytree': 0.65,
 'learning_rate': 0.005,
 'n_estimators': 2,
 'num_leaves': 16,
 'objective': 'regression',
 'random_state': 501,
 'reg_alpha': 1,
 'reg_lambda': 1,
 'subsample': 0.7}

In [46]:
prediccion = grid.predict(x_test)

In [47]:
#Estimo MAE
print(mean_absolute_error(y_test, prediccion))

1600921.908940707


In [94]:
561929.2799609166 100

SyntaxError: invalid syntax (<ipython-input-94-41a49510c087>, line 1)

# Random Search

In [68]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}
print (param_test)

{'num_leaves': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f67e04dec90>, 'min_child_samples': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f67d9161750>, 'min_child_weight': [1e-05, 0.001, 0.01, 0.1, 1, 10.0, 100.0, 1000.0, 10000.0], 'subsample': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f67d91616d0>, 'colsample_bytree': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f67da3ffa90>, 'reg_alpha': [0, 0.1, 1, 2, 5, 7, 10, 50, 100], 'reg_lambda': [0, 0.1, 1, 5, 10, 20, 50, 100]}


In [69]:
#Numero de ptos a probar 
n_HP_points_to_test = 3

import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

#n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
clf = lgb.LGBMRegressor(max_depth=-1, random_state=314, silent=True, metric='mae', n_jobs=None, n_estimators=5000)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    cv=3,
    refit=True,
    random_state=314,
    verbose=True)


In [70]:
gs.fit(x, y_train)
print('Best score with params: {} '.format( gs.best_params_))

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 10.5min finished


Best score with params: {'colsample_bytree': 0.9731668400523877, 'min_child_samples': 171, 'min_child_weight': 1e-05, 'num_leaves': 41, 'reg_alpha': 10, 'reg_lambda': 100, 'subsample': 0.5575732396028996} 


In [71]:
prediccion = gs.predict(x_test)

In [72]:
#Estimo MAE
print(mean_absolute_error(y_test, prediccion))

509839.05516658636
